# Create Training and Testing Data for a Logistic Regression 
## Version: Using Mean Patch data during Current or Future Climate
### Requires: create_UHindx_file_step1, ..file_step2, ..file_step3 (e.g., future_uh75patches_12.nc)

First, import relevant packages.

In [1]:
import xarray as xr
import numpy as np
from ncar_jobqueue import NCARCluster
from dask.distributed import Client

Choose the climate to work with (e.g., current or future).

In [29]:
which_climate = 'future'

Start dask workers with adaptive scaling to load data for training.

In [3]:
#--------------------------------------------------

#if __name__== "__main__":

#start dask workers
cluster = NCARCluster(memory="109GB", cores=36)
cluster.adapt(minimum=1, maximum=10, wait_count=60)
cluster
#print scripts
print(cluster.job_script())
#start client
client = Client(cluster)
client

#--------------------------------------------------

#!/usr/bin/env bash

#PBS -N dask-worker
#PBS -q regular
#PBS -A P54048000
#PBS -l select=1:ncpus=36:mem=109GB
#PBS -l walltime=01:00:00
#PBS -e /glade/scratch/molina/
#PBS -o /glade/scratch/molina/
JOB_ID=${PBS_JOBID%%.*}



/glade/work/molina/miniconda3/envs/python-tutorial/bin/python -m distributed.cli.dask_worker tcp://10.148.10.15:36135 --nthreads 36 --memory-limit 109.00GB --name dask-worker--${JOB_ID}-- --death-timeout 60 --local-directory /glade/scratch/molina --interface ib0



/glade/work/molina/miniconda3/envs/python-tutorial/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://10.148.10.15:36135 Dashboard: https://jupyterhub.ucar.edu/ch/user/molina/proxy/33239/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


Select climate simulation (e.g., current or future).

Load storm patch data that was previously separated using UH>75 and UH<75 m2/s2 thresholds.

In [30]:
data_dec_above = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/{which_climate}_uh75patches_12.nc", 
                                   parallel=True, combine='by_coords')
data_jan_above = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/{which_climate}_uh75patches_01.nc",
                                   parallel=True, combine='by_coords')
data_feb_above = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/{which_climate}_uh75patches_02.nc", 
                                   parallel=True, combine='by_coords')

data_mar_above = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/{which_climate}_uh75patches_03.nc", 
                                   parallel=True, combine='by_coords')
data_apr_above = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/{which_climate}_uh75patches_04.nc", 
                                   parallel=True, combine='by_coords')
data_may_above = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/{which_climate}_uh75patches_05.nc", 
                                   parallel=True, combine='by_coords')

data_above = xr.concat([data_dec_above, data_jan_above, data_feb_above, data_mar_above, data_apr_above, data_may_above], dim='patch')

In [31]:
data_dec_below = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/{which_climate}_nonuh75patches_12.nc",
                                  parallel=True, combine='by_coords')
data_jan_below = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/{which_climate}_nonuh75patches_01.nc",
                                   parallel=True, combine='by_coords')
data_feb_below = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/{which_climate}_nonuh75patches_02.nc",
                                   parallel=True, combine='by_coords')

data_mar_below = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/{which_climate}_nonuh75patches_03.nc",
                                   parallel=True, combine='by_coords')
data_apr_below = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/{which_climate}_nonuh75patches_04.nc",
                                   parallel=True, combine='by_coords')
data_may_below = xr.open_mfdataset(f"/glade/scratch/molina/WRF_CONUS1_derived/storm_envs/{which_climate}_nonuh75patches_05.nc",
                                   parallel=True, combine='by_coords')

data_below = xr.concat([data_dec_below, data_jan_below, data_feb_below, data_mar_below, data_apr_below, data_may_below], dim='patch')

Creation of various functions for use in analysis.

In [32]:
def below_permute(data_b, data_a):
    #balancing of above and below threshold data:
    #permute and slice the below threshold data to equal the above threshold data shape.
    np.random.seed(10)
    select_data = np.random.permutation(data_b.patch.shape[0])[:int(data_a.patch.shape[0])]
    train_patches = data_b.patch[select_data]
    return train_patches

def concat_files(data_uh, data_nonuh):
    #return the concatenated below and above threshold data.
    #data should be input to the function already shuffled (permuted).
    total_training_data = xr.concat([data_uh,data_nonuh],dim='patch')
    return total_training_data

def minmax_scale_apply(thedata):
    #apply min max normalize the input data
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(thedata)
    return scaler.transform(thedata)

def standardize_scale_apply(thedata):
    #standardization of the data
    #to interpret: "this data point is X standard deviations below/above the mean of the data set."
    return np.divide((thedata - np.nanmean(thedata)), np.std(thedata))

def standardize_scale_apply_test(thedata, thedatatest):
    #standardization of the test data using the training mean and standard deviation.
    return np.divide((thedatatest - np.nanmean(thedata)), np.std(thedata))

def data_permute(data, data_split=0.6):
    #permute and split the combined below/above threshold data for training and testing.
    #output train and test data.
    np.random.seed(0)
    select_train = np.random.permutation(data.shape[0])[:int(data.shape[0]*data_split)]
    np.random.seed(0)
    select_test = np.random.permutation(data.shape[0])[int(data.shape[0]*data_split):]
    train_patches = data[select_train]
    test_patches = data[select_test]
    return train_patches, test_patches

Extract the permuted below threshold data of the length of the above threshold data to balance the distribution of above and below threshold storm patches.

In [33]:
blah = below_permute(data_below, data_above)

Filter the below threshold storm patch variables, concat below and above threshold patches, take the mean of the storm patches to reduce dimensionality of data, and reshape into feature format for machine learning model.

In [34]:
mean_total_data_temp1 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).temp_sev_1.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)
mean_total_data_temp3 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).temp_sev_3.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)
mean_total_data_temp5 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).temp_sev_5.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)
mean_total_data_temp7 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).temp_sev_7.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)

mean_total_data_evwd1 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).evwd_sev_1.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)
mean_total_data_evwd3 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).evwd_sev_3.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)
mean_total_data_evwd5 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).evwd_sev_5.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)
mean_total_data_evwd7 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).evwd_sev_7.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)

mean_total_data_euwd1 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).euwd_sev_1.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)
mean_total_data_euwd3 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).euwd_sev_3.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)
mean_total_data_euwd5 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).euwd_sev_5.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)
mean_total_data_euwd7 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).euwd_sev_7.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)

mean_total_data_qvap1 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).qvap_sev_1.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)
mean_total_data_qvap3 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).qvap_sev_3.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)
mean_total_data_qvap5 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).qvap_sev_5.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)
mean_total_data_qvap7 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).qvap_sev_7.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)

mean_total_data_pres1 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).pres_sev_1.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)
mean_total_data_pres3 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).pres_sev_3.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)
mean_total_data_pres5 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).pres_sev_5.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)
mean_total_data_pres7 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).pres_sev_7.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)

mean_total_data_qgrp1 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).qgrp_sev_1.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)
mean_total_data_qgrp3 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).qgrp_sev_3.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)
mean_total_data_qgrp5 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).qgrp_sev_5.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)
mean_total_data_qgrp7 = concat_files(data_above, data_below.isel(patch=np.sort(blah))).qgrp_sev_7.mean(axis=(1,2), skipna=True).values.reshape(-1, 1)

total_result_data = np.hstack([np.ones(data_above.patch.shape[0]), np.zeros(data_below.isel(patch=np.sort(blah)).patch.shape[0])]).reshape(-1,1)

JobQueueCluster.scale_down was called with a number of worker greater than what is already running or pending.


Shuffle the data and split into training and testing data sets.

In [35]:
patch_data_scaled_training_da, patch_data_scaled_test_da = data_permute(total_result_data, data_split=0.6)

patch_data_scaled_train_temp1, patch_data_scaled_test_temp1 = data_permute(mean_total_data_temp1, data_split=0.6)
patch_data_scaled_train_temp3, patch_data_scaled_test_temp3 = data_permute(mean_total_data_temp3, data_split=0.6)
patch_data_scaled_train_temp5, patch_data_scaled_test_temp5 = data_permute(mean_total_data_temp5, data_split=0.6)
patch_data_scaled_train_temp7, patch_data_scaled_test_temp7 = data_permute(mean_total_data_temp7, data_split=0.6)

patch_data_scaled_train_evwd1, patch_data_scaled_test_evwd1 = data_permute(mean_total_data_evwd1, data_split=0.6)
patch_data_scaled_train_evwd3, patch_data_scaled_test_evwd3 = data_permute(mean_total_data_evwd3, data_split=0.6)
patch_data_scaled_train_evwd5, patch_data_scaled_test_evwd5 = data_permute(mean_total_data_evwd5, data_split=0.6)
patch_data_scaled_train_evwd7, patch_data_scaled_test_evwd7 = data_permute(mean_total_data_evwd7, data_split=0.6)

patch_data_scaled_train_euwd1, patch_data_scaled_test_euwd1 = data_permute(mean_total_data_euwd1, data_split=0.6)
patch_data_scaled_train_euwd3, patch_data_scaled_test_euwd3 = data_permute(mean_total_data_euwd3, data_split=0.6)
patch_data_scaled_train_euwd5, patch_data_scaled_test_euwd5 = data_permute(mean_total_data_euwd5, data_split=0.6)
patch_data_scaled_train_euwd7, patch_data_scaled_test_euwd7 = data_permute(mean_total_data_euwd7, data_split=0.6)

patch_data_scaled_train_qvap1, patch_data_scaled_test_qvap1 = data_permute(mean_total_data_qvap1, data_split=0.6)
patch_data_scaled_train_qvap3, patch_data_scaled_test_qvap3 = data_permute(mean_total_data_qvap3, data_split=0.6)
patch_data_scaled_train_qvap5, patch_data_scaled_test_qvap5 = data_permute(mean_total_data_qvap5, data_split=0.6)
patch_data_scaled_train_qvap7, patch_data_scaled_test_qvap7 = data_permute(mean_total_data_qvap7, data_split=0.6)

patch_data_scaled_train_pres1, patch_data_scaled_test_pres1 = data_permute(mean_total_data_pres1, data_split=0.6)
patch_data_scaled_train_pres3, patch_data_scaled_test_pres3 = data_permute(mean_total_data_pres3, data_split=0.6)
patch_data_scaled_train_pres5, patch_data_scaled_test_pres5 = data_permute(mean_total_data_pres5, data_split=0.6)
patch_data_scaled_train_pres7, patch_data_scaled_test_pres7 = data_permute(mean_total_data_pres7, data_split=0.6)

Standardize the training data.

In [36]:
data_scaled_train_temp1 = standardize_scale_apply(patch_data_scaled_train_temp1)
data_scaled_train_temp3 = standardize_scale_apply(patch_data_scaled_train_temp3)
data_scaled_train_temp5 = standardize_scale_apply(patch_data_scaled_train_temp5)
data_scaled_train_temp7 = standardize_scale_apply(patch_data_scaled_train_temp7)

data_scaled_train_evwd1 = standardize_scale_apply(patch_data_scaled_train_evwd1)
data_scaled_train_evwd3 = standardize_scale_apply(patch_data_scaled_train_evwd3)
data_scaled_train_evwd5 = standardize_scale_apply(patch_data_scaled_train_evwd5)
data_scaled_train_evwd7 = standardize_scale_apply(patch_data_scaled_train_evwd7)

data_scaled_train_euwd1 = standardize_scale_apply(patch_data_scaled_train_euwd1)
data_scaled_train_euwd3 = standardize_scale_apply(patch_data_scaled_train_euwd3)
data_scaled_train_euwd5 = standardize_scale_apply(patch_data_scaled_train_euwd5)
data_scaled_train_euwd7 = standardize_scale_apply(patch_data_scaled_train_euwd7)

data_scaled_train_qvap1 = standardize_scale_apply(patch_data_scaled_train_qvap1)
data_scaled_train_qvap3 = standardize_scale_apply(patch_data_scaled_train_qvap3)
data_scaled_train_qvap5 = standardize_scale_apply(patch_data_scaled_train_qvap5)
data_scaled_train_qvap7 = standardize_scale_apply(patch_data_scaled_train_qvap7)

data_scaled_train_pres1 = standardize_scale_apply(patch_data_scaled_train_pres1)
data_scaled_train_pres3 = standardize_scale_apply(patch_data_scaled_train_pres3)
data_scaled_train_pres5 = standardize_scale_apply(patch_data_scaled_train_pres5)
data_scaled_train_pres7 = standardize_scale_apply(patch_data_scaled_train_pres7)

Standardize the testing data using the training data distribution.

In [37]:
data_scaled_test_temp1 = standardize_scale_apply_test(patch_data_scaled_train_temp1, patch_data_scaled_test_temp1)
data_scaled_test_temp3 = standardize_scale_apply_test(patch_data_scaled_train_temp3, patch_data_scaled_test_temp3)
data_scaled_test_temp5 = standardize_scale_apply_test(patch_data_scaled_train_temp5, patch_data_scaled_test_temp5)
data_scaled_test_temp7 = standardize_scale_apply_test(patch_data_scaled_train_temp7, patch_data_scaled_test_temp7)

data_scaled_test_evwd1 = standardize_scale_apply_test(patch_data_scaled_train_evwd1, patch_data_scaled_test_evwd1)
data_scaled_test_evwd3 = standardize_scale_apply_test(patch_data_scaled_train_evwd3, patch_data_scaled_test_evwd3)
data_scaled_test_evwd5 = standardize_scale_apply_test(patch_data_scaled_train_evwd5, patch_data_scaled_test_evwd5)
data_scaled_test_evwd7 = standardize_scale_apply_test(patch_data_scaled_train_evwd7, patch_data_scaled_test_evwd7)

data_scaled_test_euwd1 = standardize_scale_apply_test(patch_data_scaled_train_euwd1, patch_data_scaled_test_euwd1)
data_scaled_test_euwd3 = standardize_scale_apply_test(patch_data_scaled_train_euwd3, patch_data_scaled_test_euwd3)
data_scaled_test_euwd5 = standardize_scale_apply_test(patch_data_scaled_train_euwd5, patch_data_scaled_test_euwd5)
data_scaled_test_euwd7 = standardize_scale_apply_test(patch_data_scaled_train_euwd7, patch_data_scaled_test_euwd7)

data_scaled_test_qvap1 = standardize_scale_apply_test(patch_data_scaled_train_qvap1, patch_data_scaled_test_qvap1)
data_scaled_test_qvap3 = standardize_scale_apply_test(patch_data_scaled_train_qvap3, patch_data_scaled_test_qvap3)
data_scaled_test_qvap5 = standardize_scale_apply_test(patch_data_scaled_train_qvap5, patch_data_scaled_test_qvap5)
data_scaled_test_qvap7 = standardize_scale_apply_test(patch_data_scaled_train_qvap7, patch_data_scaled_test_qvap7)

data_scaled_test_pres1 = standardize_scale_apply_test(patch_data_scaled_train_pres1, patch_data_scaled_test_pres1)
data_scaled_test_pres3 = standardize_scale_apply_test(patch_data_scaled_train_pres3, patch_data_scaled_test_pres3)
data_scaled_test_pres5 = standardize_scale_apply_test(patch_data_scaled_train_pres5, patch_data_scaled_test_pres5)
data_scaled_test_pres7 = standardize_scale_apply_test(patch_data_scaled_train_pres7, patch_data_scaled_test_pres7)

Stack the numpy arrays for saving.

In [38]:
X_train = np.hstack([data_scaled_train_temp1, data_scaled_train_temp3, data_scaled_train_temp5, data_scaled_train_temp7,                     
                     data_scaled_train_evwd1, data_scaled_train_evwd3, data_scaled_train_evwd5, data_scaled_train_evwd7,     
                     data_scaled_train_euwd1, data_scaled_train_euwd3, data_scaled_train_euwd5, data_scaled_train_euwd7,
                     data_scaled_train_qvap1, data_scaled_train_qvap3, data_scaled_train_qvap5, data_scaled_train_qvap7,
                     data_scaled_train_pres1, data_scaled_train_pres3, data_scaled_train_pres5, data_scaled_train_pres7
])

In [39]:
X_test = np.hstack([data_scaled_test_temp1, data_scaled_test_temp3, data_scaled_test_temp5, data_scaled_test_temp7,
                    data_scaled_test_evwd1, data_scaled_test_evwd3, data_scaled_test_evwd5, data_scaled_test_evwd7,
                    data_scaled_test_euwd1, data_scaled_test_euwd3, data_scaled_test_euwd5, data_scaled_test_euwd7,
                    data_scaled_test_qvap1, data_scaled_test_qvap3, data_scaled_test_qvap5, data_scaled_test_qvap7,
                    data_scaled_test_pres1, data_scaled_test_pres3, data_scaled_test_pres5, data_scaled_test_pres7
])

Save training and testing data for current and future climate as one file for future use.

In [40]:
data_assemble = xr.Dataset({
    'X_train':(['a','features'], X_train),
    'X_train_label':(['a'], patch_data_scaled_training_da.reshape(patch_data_scaled_training_da.shape[0])),
    'X_test':(['b','features'], X_test),
    'X_test_label':(['b'], patch_data_scaled_test_da.reshape(patch_data_scaled_test_da.shape[0])),
    },
     coords=
    {'feature':(['features'],np.array(["tk_1km", "tk_3km", "tk_5km", "tk_7km",
                                       "ev_1km", "ev_3km", "ev_5km", "ev_7km",
                                       "eu_1km", "eu_3km", "eu_5km", "eu_7km",
                                       "pr_1km", "pr_3km", "pr_5km", "pr_7km",
                                       "qv_1km", "qv_3km", "qv_5km", "qv_7km"])),
    })

In [41]:
data_assemble

<xarray.Dataset>
Dimensions:        (a: 47703, b: 31803, features: 20)
Coordinates:
    feature        (features) <U6 'tk_1km' 'tk_3km' ... 'qv_5km' 'qv_7km'
Dimensions without coordinates: a, b, features
Data variables:
    X_train        (a, features) float32 0.29431275 0.08929747 ... 0.9471039
    X_train_label  (a) float64 1.0 1.0 0.0 1.0 0.0 0.0 ... 0.0 1.0 1.0 0.0 0.0
    X_test         (b, features) float32 0.3461959 0.42525783 ... 0.42535275
    X_test_label   (b) float64 1.0 0.0 1.0 0.0 0.0 1.0 ... 1.0 0.0 0.0 0.0 0.0

In [43]:
data_assemble.to_netcdf(f"/glade/scratch/molina/WRF_CONUS1_derived/logistic_regression/{which_climate}_meanpatch_traintestdata.nc")

JobQueueCluster.scale_down was called with a number of worker greater than what is already running or pending.
